<img src='https://upload.wikimedia.org/wikipedia/fr/thumb/e/ed/Logo_Universit%C3%A9_du_Maine.svg/1280px-Logo_Universit%C3%A9_du_Maine.svg.png' width="300" height="500">
<br>
<div style="border: solid 3px #000;">
    <h1 style="text-align: center; color:#000; font-family:Georgia; font-size:26px;">Introduction à l'IA</h1>
    <p style='text-align: center;'>Master Informatique</p>
    <p style='text-align: center;'>Anhony Larcher</p>
</div>

Ce premier tutoriel est une introduction à [PyTorch](https://pytorch.org/)


# Introduction a PyTorch: les classes de base

La classe de base en **PyTorch** est le tenseur; les opérations de base sur les tenseurs sont décrites dans la [documentation de **PyTorch**](https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html)

En Pytorch il y a deux façons de définir un réseau de neurone:
* dériver la classe **module**
* utiliser un objet **sequential**

In [39]:
import numpy as np
import torch

Tout calcul en PyTorch est fait sur un *device* qu'il vous appartient de définir.

Généralement, on utilise les **cpu** pour préparer les données et visualiser, et les **gpu** pour le calcul dans les réseaux.

```python
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
```

## La classe de base pour définir un réseau de neurones est la classe **module**

On dérive cette classe pour créer sa propre architecture puis on implémente la fonction **forward**.

Nous allons définir un block résiduel (pour implémenter un réseau qui s'inspire d'un ResNet).

```python
class ResBlock(torch.nn.Module):
    """
    La fonction d'initialisation prend en entrée 
        * la dimension d'entrée (par défaut 50)
        * une liste de 2 entiers qui correspondent aux dimensions des couches cachée intermédiares.
          par défaut [128,32]
    """

    def __init__(self, à compléter):
        super(ResBlock, self).__init__()
        ... a completer
        
    def forward(self, x):
        ... a completer
```

La définition d'une classe héritée de `module` requiert l'implémentation de deux méthodes:
* ```__init__```
* `forward`
Le rôle de ces méthodes est détaillé ci-dessous.

### Initialisation
Instancie les composantes du module:
* couches neuronales
* fonctions d'activation
* couches et fonctions de régularisation (batchnorm, layernorm...)

### Méthode `Forward`
Définit le flux des données à travers les couches et fonctions instanciées dans la fonction ```__init__```.


### Retour à l'exemple du ResBlock

Afin d'implémenter votre module vous devez définir son architecture.
Dans cet exemple, le bloc doit contenir dans l'ordre:

* une première couche linear  avec:
    * un biais
* une fonction d'activation LeakyRELU
* une deuxième couche linear  avec:
    * aucun biais
* une fonction d'activation Sigmoid
* une troisième couche linear  avec:
    * un biais
    * une dimension de sortie égale à la dimension d'entrée de la première couche
* une couche de BatchNorm

L'architecture de votre ResBlock est illustrée par la figure suivante.

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'gpu'
print('Using {} device'.format(device))

Using gpu device


<img src='https://git-lium.univ-lemans.fr/cours_m2/open_resources/-/raw/master/figures/dnn_intro_1.png' width="300" height="500">

In [16]:
class ResBlock(torch.nn.Module):
    """
    La fonction d'initialisation prend en entrée 
        * la dimension d'entrée (par défaut 50)
        * une liste de 2 entiers qui correspondent aux dimensions des couches cachée intermédiares.
          par défaut [128,32]
    """

    def __init__(self, input_dim = 50,  hidden_layout = [128,32]):
        super(ResBlock, self).__init__()
        
        self.input_dim = input_dim 
        self.hidden_layout = hidden_layout

        # torch.nn.BatchNorm1d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)
        self.batchNorm = torch.nn.BatchNorm1d(input_dim)
        
        # torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)
        self.linear1 = torch.nn.Linear(input_dim, hidden_layout[0])  # ajouter biais
        self.linear2 = torch.nn.Linear(hidden_layout[0], hidden_layout[1], bias = False) 
        self.linear3 = torch.nn.Linear(hidden_layout[1], input_dim) # ajouter biais et dimension de sortie égale à l'entrée de la couche 1

        # torch.nn.LeakyReLU(negative_slope=0.01, inplace=False)
        self.leakyRELU = torch.nn.LeakyReLU()

        # torch.nn.Sigmoid(*args, **kwargs)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        out = self.linear1(x)
        out = self.leakyRELU(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        out = self.linear3(out)
        out = self.leakyRELU(out)
        out = self.batchNorm(out)
        return out



### Bonne pratique pour l'implémentation

L'implémentation de réseaux de neurones est parfois complexe.
Le nombre de couche et la modularité des réseaux nécessite de la rigueur.
Une bonne pratique, qui vous évitera un grand nombre d'erreurs consite à vérifier les dimensions d'entrée et de sortie de chaque couche afin de s'assurer que la sortie d'une couche ou d'un module peut être utilisée en entrée du suivant.

**Pensez donc à créer un batch de fausses données (au format des vraies) afin de tester votre module au fur et à mesure de son implémentation.**

Ici vous pouvez créer un batch de données aléatoires de dimension `(1, 2, 10, 100)` qui correspond à 1 batch de 1 échantillon à 2 canaux de dimension `(10, 100)`.

In [17]:
# Initialisez un ResBlock avec une entrée de dimension 20 et des dimensions cachées de 10 et 5
rb = ResBlock(20,[10,5])

In [18]:
# Create 1 batch of 100 sample of 20 dimensions
data = torch.rand(100, 20)#

In [19]:
# Test here your module
output = rb(data)

print(f"Shape of output= {output.shape}")

Shape of output= torch.Size([100, 20])


## Sequential

### Motivation

Un module est très flexible et permet d'implémenter des flux de données complexes.

Dans les cas ou le flux de données est simple et (unidirectionnel) il est possible de regrouper des couches ou traitements dans une structure séquentielle.

### Application

Ré-écrivez maintenant le module ResBlock en utilisant un objet *torch.nn.Sequential*



In [28]:
class ResBlock2(torch.nn.Module):
    """
    La fonction d'initialisation prend en entrée 
        * la dimension d'entrée (par défaut 50)
        * une liste de 2 entiers qui correspondent aux dimensions des couches cachée intermédiares.
          par défaut [128,32]
    """

  
    def __init__(self, input_dim = 50,  hidden_layout = [128,32]):
        super(ResBlock2, self).__init__()
        
        self.input_dim = input_dim 
        self.hidden_layout = hidden_layout

        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_layout[0]),
                            torch.nn.LeakyReLU(),
                            torch.nn.Linear(hidden_layout[0], hidden_layout[1], bias = False),
                            torch.nn.Sigmoid(),
                            torch.nn.Linear(hidden_layout[1], input_dim),
                            torch.nn.LeakyReLU(),
                            torch.nn.BatchNorm1d(input_dim))

            
    def forward(self, x):
        return self.sequential(x)

In [30]:
# Test here your module
rb2 = ResBlock2(20,[10,5])
output2 = rb2(data)

print(f"Shape of output= {output2.shape}")
rb2.forward(data)

Shape of output= torch.Size([100, 20])


tensor([[-0.0289, -0.0187,  0.0689,  ...,  1.6605,  0.0207,  1.3122],
        [ 0.0431,  0.0310,  0.0239,  ..., -0.2866,  0.0346, -0.3346],
        [ 0.0085,  0.0299,  0.0490,  ..., -0.3898,  0.0112, -1.0373],
        ...,
        [-0.0410, -0.0118,  0.0046,  ...,  1.0469, -0.0062,  0.5809],
        [-0.0088,  0.0027,  0.0306,  ...,  0.1713,  0.0060, -0.0658],
        [-0.0127,  0.0072,  0.0370,  ...,  0.7858,  0.0094,  0.1631]],
       grad_fn=<NativeBatchNormBackward0>)

# Retour sur les modules

La classe module est très flexible et permet de créer des **pipelines** complexes comme celui de la figure ci-dessous que nous allons implémenter.

<img src='https://git-lium.univ-lemans.fr/cours_m2/open_resources/-/raw/master/figures/dnn_intro_2.png' width="300" height="500">

In [32]:
class ResNet(torch.nn.Module):
    """

    """
    def __init__(self, input_size=50, hiddens=[128, 64]):
        super(ResNet, self).__init__()

        self.resBlock1 = ResBlock(input_size,hiddens)

        self.resBlock2 = ResBlock2(input_size, hiddens)
    
    def forward(self, x):
        identity = x
        F_X = self.resBlock1(x)
        F_F_X = self.resBlock2(F_X)
        return identity + F_X + F_F_X

In [48]:
# Instanciez un ResNet
nnet = ResNet(20,[10,5])

In [49]:
# Créez vos données pour vérifier l'implementation de votre ResNet et testez
output = nnet(data)

In [50]:
print(f"Shape of output= {output.shape}")
nnet.forward(data)

Shape of output= torch.Size([100, 20])


tensor([[ 0.5075,  0.3821,  0.6690,  ..., -0.4473,  0.7991,  1.2656],
        [ 0.2378, -0.2155,  0.2120,  ...,  0.1371,  0.4747,  0.4346],
        [ 0.8827,  0.1168,  0.6713,  ..., -0.3406,  0.1622, -1.4847],
        ...,
        [ 0.3657, -0.2572,  0.5725,  ...,  0.5019,  0.2576,  1.1354],
        [ 0.6937,  0.2434, -0.0081,  ...,  0.3436,  0.7878, -0.0636],
        [ 0.2952, -0.0886,  0.2412,  ...,  1.2565,  0.2767,  0.8529]],
       grad_fn=<AddBackward0>)

# Datasets, Dataloaders et Datasamplers

Les DataSets sont les classes qui permettent de charger et préparer les exemples.

Les dataloaders permettent de gérer les batchs

Les DataSamplers permettent de définir précisément les données fournies au réseau à chaque époque (équilibrage des batchs, ordre...)


In [38]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import Sampler

Implémentez un `DataSet` qui génère une liste de 128 vecteurs de dimension $50$ dont les éléments suivent une distribution normale.
Lorsque la méthode `__getitem__` est appelée, le `DataSet` retourne la matrice correspondante.

In [42]:
class RandomDataset(Dataset):
    def __init__(self):
        """
        de nombreuses transformations existent qui permettent d'augmenter les données simplement
        """
        self.data = np.random.randn(128*50).reshape(128,50)
        self.labels = np.random.randint(0,10,128)
        
        self.len = len(self.labels)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [43]:
# Instanciez et testez votre RandomDataset en affichant le 3e exemple généré
training_set = RandomDataset()
training_set.__getitem__(2)
print(f"dimension des données: {training_set.__getitem__(2)[0].shape}")
print(f"label associé à cet échantillon: {training_set.__getitem__(2)[1]}")

dimension des données: (50,)
label associé à cet échantillon: 1


In [35]:
# Créez un DataLoader qui utilise votre RandomDataset pour générer des batchs de 16 échantillons tirés dans un ordre aléatoire.
# Attention il est possible que l'environnement du notebook ne vous permette pas de lancer ce DataLoader.
# Si c'est le cas, pourquoi?

# Pour résoudre ce problème recopiez le code du RandomDataset dans un module python externe que vous importerez ensuite.


In [36]:
# Affichez le nombre de batchs générés par votre DataLoader
len(training_loader)

8

In [ ]:
# Testez ce DataLoader


## DataSampler

Implementez un data sampler qui, parmi les 128 exemples du dataset, sélectionne ces échantillons aléatoirement

In [19]:
class BatchSampler(torch.utils.data.Sampler):
    """
    Data Sampler used to generate uniformly distributed batches
    """


Utilisez votre `DataSampler` avec un DataLoader qui n'effectue pas de shuffle sur vos données et vérifier son fonctionnement.

In [38]:
from randomset import RandomDataset2
training_set = RandomDataset2()
sampler = BatchSampler(16)

training_loader = DataLoader(training_set,
                             batch_size=16,
                             shuffle=False,
                             drop_last=True,
                             pin_memory=True,
                             #sampler=sampler,
                             num_workers=1,
                             persistent_workers=False)

In [39]:
# Testez ce DataLoader
for batch_idx, (data, target) in enumerate(training_loader):
    if batch_idx == 1:
        print(data)

tensor([[[-0.7937,  1.4512, -0.4704,  ...,  0.0145, -0.2925,  0.9257],
         [-0.0340, -0.0792,  0.0186,  ...,  0.1165, -1.1145, -0.8483],
         [-3.5411,  0.4113, -1.2204,  ...,  1.1191, -0.2236,  0.1004],
         ...,
         [-0.7605,  0.1050,  1.8894,  ...,  0.5267,  1.4374, -0.3940],
         [ 0.3630, -0.8052,  0.9325,  ...,  0.7135,  0.6261,  0.4511],
         [ 1.2429, -0.8078, -0.5532,  ...,  0.2315, -0.7555, -0.1166]],

        [[-1.8358,  1.1586,  0.8221,  ...,  0.7761,  3.3134,  1.5142],
         [ 0.6716,  0.7796, -1.6265,  ..., -1.6118, -1.1040, -0.3088],
         [ 0.0318,  1.0291,  0.9390,  ...,  3.0728,  0.1594,  0.8606],
         ...,
         [ 0.4851,  1.0680,  1.5948,  ...,  0.2656, -0.1093,  0.3473],
         [ 1.0904,  0.6719, -0.9522,  ..., -0.9245,  0.5239, -0.9447],
         [-0.5843,  0.8847,  0.3747,  ...,  0.3761,  1.5945, -0.5665]],

        [[ 1.4093, -0.5195,  0.6137,  ..., -1.1289, -0.4026, -0.4182],
         [ 1.2400,  2.1219, -1.1617,  ..., -0

# Save and Load models

It is advised to save and load models to use the **load_state_dict** method.

Beware, saving and loading models requires to know on which device the model is loaded.

*Question: How to load a model on a selected device?*

In [ ]:
# Save and Load your model here
...

# Make PyTorch deterministic

Randomness is very present in many steps of deep learning.

In order to come closer to reproducibility you need to :

In [ ]:
import numpy
import random
import torch

numpy.random.seed(0)
random.seed(0)
torch.manual_seed(0)

Convolution can involve the choice of different algorithms.

**When a cuDNN convolution is called with a new set of size parameters, an optional feature can run multiple convolution algorithms, benchmarking them to find the fastest one.**

Disabling the benchmarking feature with ```torch.backends.cudnn.benchmark = False``` causes cuDNN to deterministically select an algorithm, possibly at the cost of reduced performance.

However, if you do not need reproducibility across multiple executions of your application, then performance might improve if the benchmarking feature is enabled with ```torch.backends.cudnn.benchmark = True```.


In [ ]:
torch.use_deterministic_algorithms(True)

DataLoader will reseed workers following Randomness in multi-process data loading algorithm. 

Use ```worker_init_fn()``` and generator to preserve reproducibility: